### Handle course data update

In [1]:
current_term = "2022-23 Term 2"

#### Fetch data

In [2]:
from scraper import CourseScraper
import os

# To merge old courses
old_data_dir = os.path.abspath("../data")

cs = CourseScraper(merge_dir=old_data_dir, current_term=current_term)
timestamp = cs.parse_all(skip_parsed=True, verbose=False)

Parsing courses for all 251 subjects, skip if already existed


100%|██████████| 251/251 [00:23<00:00, 10.51it/s]

Done! Saved at data/1666356456


#### Derive data

In [3]:
cs.post_processing(stat=True)
cs.info()

Found 2958 instructors
Number of departments: 125
Number of courses: 8414


#### Update cutopia-data

In [4]:
import os

p = os.system(f'sh move_data.sh {timestamp}')


moving 1666356456 to /home/mike/Github/CUtopia/data


+ git add .
+ git commit -am Update data @1666356456


[master 6e5c16f] Update data @1666356456
 2 files changed, 2 insertions(+), 2 deletions(-)
 rewrite resources/instructors.json (99%)


+ git push origin master
To https://github.com/cutopia-lab/cuhk-course-data
   82ec002..6e5c16f  master -> master


#### Update FE & BE

In [5]:
import time
from dotenv import set_key

deploy_duration = 15 # in mins (need to deploy both fe & be within the duration)
deploy_duration_seconds = deploy_duration * 60 # in mins

now = str(int((time.time() + deploy_duration_seconds) * 1000))

# Update FE
os.system('yarn loadnmount')
set_key('../frontend/.env', 'REACT_APP_LAST_DATA_UPDATE', str(now))
set_key('../frontend/.env', 'REACT_APP_CURRENT_TERM', current_term)

# Update BE
os.system('sh ../backend/tools/copy-data.sh')
print(f'Updated, please deploy in {deploy_duration} mins')

yarn run v1.22.17
$ yarn load-data && yarn mount-data
$ git submodule update --init --remote
$ yarn fe move-data && yarn be move-data
$ yarn workspace cutopia-frontend move-data
$ cp -R ../data/resources public/ && cp -r ../data/derived/faculty_subjects.json src/constants/faculty_subjects.json
$ yarn workspace cutopia-backend move-data
$ bash tools/copy-data.sh
Done in 3.24s.
Updated, please deploy in 15 mins
